In [1]:
# Arthor : Wang Chang
# zID: z5196324
# Finished by 08/03/20
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import time

In [2]:
def sad(window1, window2):
    return np.sum(np.absolute(window1 - window2))
def ssd(window1, window2):
    return np.sum(np.power(window1 - window2, 2))

In [3]:
def get_roi(img, h_factor, w_factor, window, threshold, step):
    height, width = img.shape
    h_start = int(h_factor * height)
    h_end = h_start + threshold * step + window - 1
    w_start = int(w_factor * width)
    w_end = w_start + threshold * step + window - 1
    
    return img[h_start:h_end, w_start:w_end]

In [4]:
def normalization(img):
    return cv2.normalize(img, 0, 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

In [5]:
def get_best_match(img1, img2, window, step):
    height, width = img1.shape
    num_window_h = max((height - window) / step,0) + 1
    num_window_w = max((width - window) / step,0) + 1
    
    #init
    img1_h_best, img1_w_best, img2_h_best, img2_w_best = 0, 0, 0, 0
    window1 = img1[0:window, 0:window]
    window2 = img2[0:window, 0:window]
    best_match = ssd(window1, window2)
    for  img1_h in range(0, height - window + 1, step):
        for img1_w in range(0, width - window + 1, step):
            window1 = img1[img1_h:(img1_h + window), img1_w:(img1_w + window)]
            for  img2_h in range(0, height - window + 1, step):
                for img2_w in range(0, width - window + 1, step):
                    window2 = img2[img2_h:(img2_h + window), img2_w:(img2_w + window)]
                    curr_match = ssd(window1, window2)
                    if curr_match < best_match:
                        best_match = curr_match
                        img1_h_best, img1_w_best = img1_h, img1_w
                        img2_h_best, img2_w_best = img2_h, img2_w
                        
    return img1_h_best, img1_w_best, img2_h_best, img2_w_best, best_match

In [6]:
def task1(img_name, h_factor, w_factor, window, threshold, step):
    start_time = time.clock()
    img = cv2.imread(img_name)
    height, width = img.shape[:2]
    
    #split image
    sub_height = int(height/3)
    blue = img[0:sub_height, :, 0]
    green = img[sub_height:sub_height * 2, :, 1]
    red = img[sub_height * 2:sub_height * 3, :, 2]
    
    #scale the parameters
    scale_factor = max(int(min(sub_height / 350, width / 400)), 1)
    window = window * scale_factor
    step = step * scale_factor
    
    #get roi
    blue_roi = get_roi(blue, h_factor, w_factor, window, threshold, step)
    green_roi = get_roi(green, h_factor, w_factor, window, threshold, step)
    red_roi = get_roi(red, h_factor, w_factor, window, threshold, step)
    
    #normalization, to avoid image too bright or dark, and the strange parameters
    normalized_blue = normalization(blue_roi)
    normalized_green = normalization(green_roi)
    normalized_red = normalization(red_roi)
    
    #get best match of green based on blue
    rows, cols = blue.shape
    b_h, b_w, g_h, g_w, best_match = get_best_match(normalized_blue, normalized_green, window, step)
    #print('match blue and green :', str([b_h, b_w, g_h, g_w]))
    changed_green = cv2.warpAffine(green, np.float32([[1,0,b_w - g_w], [0,1,b_h - g_h]]), (cols, rows))
    
    #get best match of red based on blue
    b_h, b_w, r_h, r_w, best_match = get_best_match(normalized_blue, normalized_red, window, step)
    #print('match blue and red :', str([b_h, b_w, r_h, r_w]))
    changed_red = cv2.warpAffine(red, np.float32([[1,0,b_w - r_w], [0,1,b_h - r_h]]), (cols, rows))
    
    #merge img
    merged_img = cv2.merge([blue, changed_green, changed_red])
    end_time = time.clock()
    print('running time : ' + str(end_time - start_time))
    return merged_img

In [7]:

merged_img1 = task1('s1.jpg', 0.60, 0.40, 40, 11, 1)
merged_img2 = task1('s2.jpg', 0.50, 0.50, 40, 12, 1)
merged_img3 = task1('s3.jpg', 0.50, 0.75, 30, 15, 1)
merged_img4 = task1('s4.jpg', 0.50, 0.50, 30, 10, 1)
merged_img5 = task1('s5.jpg', 0.50, 0.50, 30, 10, 1)

running time : 1.8292500427654153
running time : 2.3302335485497094
running time : 3.941033899838459
running time : 0.7934321803724718
running time : 0.732691463628063


In [77]:
cv2.imwrite('task1-img1.jpg', merged_img1)
cv2.imwrite('task1-img2.jpg', merged_img2)
cv2.imwrite('task1-img3.jpg', merged_img3)
cv2.imwrite('task1-img4.jpg', merged_img4)
cv2.imwrite('task1-img5.jpg', merged_img5)

True

In [8]:
def get_best_match_2(img1, img2, h_new1, w_new1, h_new2, w_new2, window, step):
    #scale the parameters
    height, width = img1.shape
    scale_factor = max(int(min(height / 350, width / 400)), 1)
    window = window * scale_factor
    step= step * scale_factor
    #init the result
    window1 = img1[h_new1 : h_new1 + window, w_new1 : w_new1 + window]
    window2 = img2[h_new2 : h_new2 + window, w_new2 : w_new2 + window]
    img1_h_best, img1_w_best, img2_h_best, img2_w_best = h_new1, w_new1, h_new2, w_new2
    best_match = ssd(window1, window2)
    #general other candidates, the 8 unit around
    candidates = [[h_new1 - step, w_new1 - step, h_new2 - step, w_new2 - step], \
                  [h_new1 - step, w_new1 + step, h_new2 - step, w_new2 + step], \
                  [h_new1 - step, w_new1, h_new2 - step, w_new2 - step], \
                  [h_new1, w_new1 - step, h_new2, w_new2 - step], \
                  [h_new1, w_new1 + step, h_new2, w_new2 + step],\
                  [h_new1 + step, w_new1 - step, h_new2 + step, w_new2 - step], \
                  [h_new1 + step, w_new1 + step, h_new2 + step, w_new2 + step], \
                  [h_new1 + step, w_new1, h_new2 + step, w_new2]]
    #finding best match
    for i in candidates[:]:
        h_candidate1, w_candidate1, h_candidate2, w_candidate2 = i
        if h_candidate1 < 0 or h_candidate1 + window >= height or h_candidate2 < 0 or h_candidate2 + window >= height or w_candidate1 < 0 or w_candidate1 + window >= width or w_candidate2 < 0 or w_candidate2 + window >= width:
            continue
        window1 = img1[h_candidate1:h_candidate1 + window, w_candidate1:w_candidate1 + window]
        window2 = img2[h_candidate2:h_candidate2 + window, w_candidate2:w_candidate2 + window]
        curr_match = ssd(window1, window2)
        if curr_match < best_match:
            best_match = curr_match
            img1_h_best, img1_w_best, img2_h_best, img2_w_best = h_candidate1, w_candidate1, h_candidate2, w_candidate2
    return img1_h_best, img1_w_best, img2_h_best, img2_w_best, best_match

In [9]:
def pyramid(img1, img2, sharpen_factor, h_factor, w_factor, window, threshold, step):
    img1_down1 = cv2.pyrDown(img1)
    img2_down1 = cv2.pyrDown(img2)
    
    img1_down2 = cv2.pyrDown(img1_down1)
    img2_down2 = cv2.pyrDown(img2_down1)
    
    # get roi
    img1_down2_roi = get_roi(img1_down2, h_factor, w_factor, window, threshold, step)
    img2_down2_roi = get_roi(img2_down2, h_factor, w_factor, window, threshold, step)
    
    # get best match
    h1,w1,h2,w2,best_match = get_best_match(img1_down2_roi, img2_down2_roi, window, step)
    h1,w1,h2,w2,best_match = get_best_match_2(img1_down1, img2_down1, h1*2, w1*2, h2*2, w2*2, window, step)
    h1,w1,h2,w2,best_match = get_best_match_2(img1, img2, h1*2, w1*2, h2*2, w2*2, window, step)
    
    return h1, w1, h2, w2, best_match

In [10]:
def task2(img_name, sharpen_factor, h_factor, w_factor, window, threshold, step):
    start_time = time.clock()
    img = cv2.imread(img_name)
    
    # split the img
    sub_height = int(img.shape[0] / 3)
    blue = img[0:sub_height,:,0]
    green = img[sub_height:sub_height*2,:,1]
    red = img[sub_height*2:sub_height*3,:,2]
    
    # normalization
    normalized_blue = normalization(blue)
    normalized_green = normalization(green)
    normalized_red = normalization(red)
    
    #get best match of green based on blue
    rows, cols = blue.shape
    b_h, b_w, g_h, g_w, best_match = pyramid(normalized_blue, normalized_green, sharpen_factor, h_factor, w_factor, window, threshold, step)
    #print('match blue and green :', str([b_h, b_w, g_h, g_w]))
    changed_green = cv2.warpAffine(green, np.float32([[1,0,b_w - g_w], [0,1,b_h - g_h]]), (cols, rows))
    
    #get best match of red based on blue
    b_h, b_w, r_h, r_w, best_match = pyramid(normalized_blue, normalized_red, sharpen_factor, h_factor, w_factor, window, threshold, step)
    #print('match blue and red :', str([b_h, b_w, r_h, r_w]))
    changed_red = cv2.warpAffine(red, np.float32([[1,0,b_w - r_w], [0,1,b_h - r_h]]), (cols, rows))
    
    merged_img = cv2.merge([blue, changed_green, changed_red])
    end_time = time.clock()
    print('overall time : ' + str(end_time - start_time))
    return merged_img

In [11]:
merged_img11 = task2('s1.jpg', 20, 0.8, 0, 50, 15, 1)
merged_img22 = task2('s2.jpg', 20, 0.8, 0, 30, 15, 1)
merged_img33 = task2('s3.jpg', 20, 0.8, 0, 20, 15, 1)
merged_img44 = task2('s4.jpg', 20, 0.8, 0, 40, 10, 1)
merged_img55 = task2('s5.jpg', 20, 0.8, 0.5, 50, 10, 1)

overall time : 0.016256783701024347
overall time : 0.016067154146309548
overall time : 0.016944585898434283
overall time : 0.013579846486974745
overall time : 0.014733031216580983


In [78]:
cv2.imwrite('task2-img1.jpg', merged_img11)
cv2.imwrite('task2-img2.jpg', merged_img22)
cv2.imwrite('task2-img3.jpg', merged_img33)
cv2.imwrite('task2-img4.jpg', merged_img44)
cv2.imwrite('task2-img5.jpg', merged_img55)

True

In [12]:
merged_img111 = task2('00549u.jpg', 20, 0.8, 0, 50, 15, 1)

overall time : 8.569026145569918


In [23]:
cv2.imwrite('task2-high-pixel 1.jpg', merged_img111)

True

In [13]:
merged_img222 = task2('00911u.jpg', 20, 0.8, 0, 50, 15, 1)

overall time : 7.795815191776718


In [25]:
cv2.imwrite('task2-high-pixel 2.jpg', merged_img222)

True

In [14]:
def cut(img, lines, int_upper, int_lower, int_left, int_right):
    height, width = img.shape[:2]
    left_vertical, right_vertical, upper_horizontal, lower_horizontal = 0, width - 1, 0,height - 1
    for rho, theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = int(a * rho)
        y0 = int(b * rho)
        if a > b:
            if x0 < width / 2:
                left_vertical = max(x0, left_vertical)
            else:
                right_vertical = min(x0, right_vertical)
        else:
            if y0 < width / 2:
                upper_horizontal = max(y0, upper_horizontal)
            else:
                lower_horizontal = min(y0, lower_horizontal)
    return img[(upper_horizontal + int_upper):(lower_horizontal - int_lower), (left_vertical + int_left):(right_vertical - int_right)]

In [15]:
#remove the border
def remove_border(img, kernel_size = 5, canny_min = 50, canny_max = 150, int_upper = 10, int_lower = 20, int_left = 15, int_right = 15):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    edged_img = cv2.Canny(gray, canny_min, canny_max)
    lines = cv2.HoughLines(edged_img, 1, np.pi/180, 200)
    cut_img = cut(img, lines, int_upper, int_lower, int_left, int_right)
    return cut_img
    
cut_img = remove_border(merged_img1, kernel_size = 5, canny_min = 50, canny_max = 150, int_upper = 10, int_lower = 20, int_left = 15, int_right = 15)


In [16]:
cv2.imwrite('task3-cut-1.jpg', cut_img)

True

In [26]:
def contrast(img, factor):
    img_pil = Image.fromarray(img)
    img_enhanced = ImageEnhance.Contrast(img_pil).enhance(factor)
    img_cv = np.array(img_enhanced, dtype = np.uint8)
    return img_cv
contrast_img = contrast(cut_img, 1.5)


In [27]:
cv2.imwrite('task3-contrast.jpg', contrast_img)

True

In [28]:
def brightness(img, factor):
    img_pil = Image.fromarray(img)
    img_enhanced = ImageEnhance.Brightness(img_pil).enhance(factor)
    img_cv = np.array(img_enhanced, dtype = np.uint8)
    return img_cv
brightness_img = contrast(contrast_img, 1.5)

In [29]:
cv2.imwrite('task3-brightness.jpg', brightness_img)

True

In [30]:
def sharpness(img, factor):
    img_pil = Image.fromarray(img)
    img_enhanced = ImageEnhance.Sharpness(img_pil).enhance(factor)
    img_cv = np.array(img_enhanced, dtype = np.uint8)
    return img_cv
sharpness_img = contrast(brightness_img, 1.5)

In [31]:
cv2.imwrite('task3-sharpness.jpg', sharpness_img)

True